In [1]:
import cv2



hog = cv2.HOGDescriptor()


print(hog.blockSize)
print(hog.blockStride)
print(hog.nbins)
print(hog.cellSize)
print(hog.winSize)





(16, 16)
(8, 8)
9
(8, 8)
(64, 128)


[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]


In [87]:
import cv2
import numpy as np

def my_hog(img):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    
    bin_n = 16

    # quantizing binvalues in (0...16)
    bins = np.int32(bin_n*ang/(2*np.pi))

    # Divide to 4 sub-squares
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)
    norma = np.sum(hist * hist)
    hist = hist / np.sqrt(norma)
    return hist

img = cv2.imread('Merged/18.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

my_hog(img)    
    
    
    
    
    
    

array([ 0.01285062,  0.00122872,  0.        ,  0.00068688,  0.01006214,
        0.        ,  0.00245745,  0.00137375,  0.00347535,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.05140247,  0.0079867 ,  0.        ,  0.00274751,
        0.03923873,  0.        ,  0.0079867 ,  0.00274751,  0.04271408,
        0.0079867 ,  0.        ,  0.00274751,  0.04651239,  0.        ,
        0.01167287,  0.00480814,  0.05899908,  0.01535904,  0.        ,
        0.00686877,  0.05208934,  0.        ,  0.00921542,  0.00343439,
        0.07120379,  0.00921542,  0.        ,  0.00343439,  0.02812579,
        0.        ,  0.00675798,  0.00206063,  0.49175781,  0.07372339,
        0.        ,  0.02953572,  0.4683992 ,  0.        ,  0.0847819 ,
        0.03571761,  0.47401711,  0.07618084,  0.        ,  0.03090947,
        0.51325586,  0.        ,  0.08109573,  0.03365698])

In [81]:
import csv
import cv2
import numpy as np


blockSize = (8, 8)
blockStride = (4, 4)
nbins = 4
cellSize = (4, 4)
winSize = (8, 16)
derivAperture = 1
winSigma = 4.0
histogramNormType = 0
L2HysThreshold = 2.0000000000000001e-01
gammaCorrection = 0
nlevels = 32
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins, derivAperture, winSigma, histogramNormType, L2HysThreshold, gammaCorrection, nlevels)

with open('train_labels.csv') as labels:
    reader = csv.DictReader(labels)
    img_files = []
    y_labels = []
    for row in reader:
        img_files.append('./Train/' + row['category'] + '/' + row['id'])
        y_labels.append(int(row['category']))

print(len(y_labels), len(img_files))


7000 7000


In [88]:
x_data = []

for filename in img_files:
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hog_desc = my_hog(img)
    x_data.append(hog_desc)
    
print(len(x_data))

7000


In [89]:
from sklearn import svm

clf = svm.SVC()
clf.fit(x_data, y_labels)



SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [95]:
import os
import pandas as pd
test_files = os.listdir('Merged')
print(len(test_files))
print(test_files[:10])

descriptors = []

for filename in test_files:
    img = cv2.imread('Merged/' + filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    descriptors.append(my_hog(img))

label = clf.predict(descriptors)

df = pd.DataFrame()
df['fname'] = test_files
df['class'] = label
df.to_csv('result.csv', index=False, header=True)



1750
['0.png', '0.png.png', '10079.png', '1018.png', '10272.png', '10317.png', '10317.png.png', '10368.png', '10368.png.png', '10428.png']
